In [11]:
import torch

paths = [
    "FL_Models/federated_model.pth",
    "FL_Models/federated_model2.pth",
    "FL_Models/federated_model3.pth"
]

for path in paths:
    try:
        state = torch.load(path, map_location='cpu')
        if isinstance(state, dict) and "state_dict" in state:
            state = state["state_dict"]
        print(f"\n✅ {path} loaded:")
        print(f"   Keys: {list(state.keys())[:5]} ... Total: {len(state.keys())}")
    except Exception as e:
        print(f"\n❌ Error loading {path}: {e}")



✅ FL_Models/federated_model.pth loaded:
   Keys: ['conv1.weight', 'conv1.bias', 'bn3.weight', 'bn3.bias', 'bn3.running_mean'] ... Total: 17

✅ FL_Models/federated_model2.pth loaded:
   Keys: ['conv1.weight', 'conv1.bias', 'bn3.weight', 'bn3.bias', 'bn3.running_mean'] ... Total: 17

✅ FL_Models/federated_model3.pth loaded:
   Keys: ['conv1.weight', 'conv1.bias', 'bn3.weight', 'bn3.bias', 'bn3.running_mean'] ... Total: 17


C:\Users\Usuario\AppData\Local\Temp\ipykernel_24076\1885524666.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(path, map_location='cpu')


In [24]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import numpy as np
from model import Net  # Asegúrate de que el modelo corregido esté aquí

# ----------- CONFIGURACIÓN -------------
MODEL_PATH = "federated_m3.pth"
TEST_DATA_PATH = "../Data/Testing"
BATCH_SIZE = 32
IMAGE_SIZE = (64, 64)  # Debe coincidir con lo usado en entrenamiento
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------- CARGA DEL MODELO -------------
model = Net(num_classes=4)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# ----------- TRANSFORMACIONES -------------
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# ----------- CARGA DE DATOS DE PRUEBA -------------
test_dataset = datasets.ImageFolder(root=TEST_DATA_PATH, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ----------- EVALUACIÓN -------------
criterion = nn.CrossEntropyLoss()
total_loss = 0
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# ----------- RESULTADOS -------------
accuracy = 100 * correct / total
avg_loss = total_loss / len(test_loader)

print("\n🧪 Resultados de evaluación:")
print(f"   - Accuracy: {accuracy:.2f}%")
print(f"   - AVG Loss: {avg_loss:.4f}")
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))


C:\Users\Usuario\AppData\Local\Temp\ipykernel_24076\390845853.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=


🧪 Resultados de evaluación:
   - Accuracy: 95.39%
   - AVG Loss: 0.2238

📊 Classification Report:
              precision    recall  f1-score   support

      glioma       0.98      0.94      0.96       310
  meningioma       0.91      0.90      0.90       306
     notumor       0.96      0.99      0.97       465
   pituitary       0.96      0.98      0.97       330

    accuracy                           0.95      1411
   macro avg       0.95      0.95      0.95      1411
weighted avg       0.95      0.95      0.95      1411

